## Import into colmap and run reconstruction

In [ ]:
import os, argparse, h5py, warnings
import numpy as np
from tqdm import tqdm
from PIL import Image, ExifTags
import pycolmap
from database import COLMAPDatabase, image_ids_to_pair_id

def get_focal(image_path, err_on_default=False):
    image         = Image.open(image_path)
    max_size      = max(image.size)

    exif = image.getexif()
    focal = None
    if exif is not None:
        focal_35mm = None
        # https://github.com/colmap/colmap/blob/d3a29e203ab69e91eda938d6e56e1c7339d62a99/src/util/bitmap.cc#L299
        for tag, value in exif.items():
            focal_35mm = None
            if ExifTags.TAGS.get(tag, None) == 'FocalLengthIn35mmFilm':
                focal_35mm = float(value)
                break

        if focal_35mm is not None:
            focal = focal_35mm / 35. * max_size
    
    if focal is None:
        if err_on_default:
            raise RuntimeError("Failed to find focal length")

        # failed to find it in exif, use prior
        FOCAL_PRIOR = 1.2
        focal = FOCAL_PRIOR * max_size

    return focal

def create_camera(db, image_path, camera_model):
    image         = Image.open(image_path)
    width, height = image.size

    focal = get_focal(image_path)

    if camera_model == 'simple-pinhole':
        model = 0 # simple pinhole
        param_arr = np.array([focal, width / 2, height / 2])
    if camera_model == 'pinhole':
        model = 1 # pinhole
        param_arr = np.array([focal, focal, width / 2, height / 2])
    elif camera_model == 'simple-radial':
        model = 2 # simple radialF
        param_arr = np.array([focal, width / 2, height / 2, 0.1])
    elif camera_model == 'opencv':
        model = 4 # opencv
        param_arr = np.array([focal, focal, width / 2, height / 2, 0., 0., 0., 0.])
         
    return db.add_camera(model, width, height, param_arr)


def add_keypoints(db, h5_path, image_path, img_ext, camera_model, single_camera = True):
    keypoint_f = h5py.File(os.path.join(h5_path, 'keypoints.h5'), 'r')

    camera_id = None
    fname_to_id = {}
    for filename in tqdm(list(keypoint_f.keys())):
        keypoints = keypoint_f[filename][()]
        # print(f'Adding keypoints for {filename}')
        fname_with_ext = filename + img_ext
        # print(fname_with_ext)

        path = os.path.join(image_path, fname_with_ext)
        if not os.path.isfile(path):
            raise IOError(f'Invalid image path {path}')

        if camera_id is None or not single_camera:
            camera_id = create_camera(db, path, camera_model)
        image_id = db.add_image(fname_with_ext, camera_id)
        fname_to_id[filename] = image_id

        db.add_keypoints(image_id, keypoints)

    return fname_to_id

def add_matches(db, h5_path, fname_to_id):
    match_file = h5py.File(os.path.join(h5_path, 'matches.h5'), 'r')
    
    added = set()
    n_keys = len(match_file.keys())
    n_total = (n_keys * (n_keys - 1)) // 2

    with tqdm(total=n_total) as pbar:
        for key_1 in match_file.keys():
            group = match_file[key_1]
            for key_2 in group.keys():
                id_1 = fname_to_id[key_1]
                id_2 = fname_to_id[key_2]

                pair_id = image_ids_to_pair_id(id_1, id_2)
                if pair_id in added:
                    warnings.warn(f'Pair {pair_id} ({id_1}, {id_2}) already added!')
                    continue
            
                matches = group[key_2][()]
                db.add_matches(id_1, id_2, matches)

                added.add(pair_id)

                pbar.update(1)

In [ ]:
feature_dir = 'features'
dirname = 'dirname'


def import_into_colmap(img_dir,
                       feature_dir ='.featureout',
                       database_path = 'colmap.db',
                       img_ext='.jpg'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, img_ext, 'simple-radial', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )

    db.commit()
    return

database_path = 'colmap.db'
!rm {database_path}
import_into_colmap(dirname,feature_dir=feature_dir, database_path=database_path)

100%|██████████| 83/83 [00:00<00:00, 323.74it/s]
3403it [00:00, 5752.01it/s]                          


In [30]:
output_path = 'colmap'

In [31]:
pycolmap.match_exhaustive(database_path)

if not os.path.isdir(output_path):
    os.makedirs(output_path)

maps = pycolmap.incremental_mapping(database_path, dirname, output_path)
maps[0].write(output_path)

I20241112 13:45:49.256878 222839 misc.cc:198] 
Exhaustive feature matching
I20241112 13:45:49.269016 222839 feature_matching.cc:231] Matching block [1/2, 1/2]
I20241112 13:46:07.159574 222839 feature_matching.cc:46]  in 17.891s
I20241112 13:46:07.217865 222839 feature_matching.cc:231] Matching block [1/2, 2/2]
I20241112 13:46:18.601106 222839 feature_matching.cc:46]  in 11.383s
I20241112 13:46:18.604779 222839 feature_matching.cc:231] Matching block [2/2, 1/2]
I20241112 13:46:37.671049 222839 feature_matching.cc:46]  in 19.066s
I20241112 13:46:37.695986 222839 feature_matching.cc:231] Matching block [2/2, 2/2]
I20241112 13:46:47.358383 222839 feature_matching.cc:46]  in 9.662s
I20241112 13:46:47.397177 222839 timer.cc:91] Elapsed time: 0.970 [minutes]
I20241112 13:46:47.817788 222879 misc.cc:198] 
Loading database
I20241112 13:46:47.820967 222879 database_cache.cc:54] Loading cameras...
I20241112 13:46:47.821341 222879 database_cache.cc:64]  83 in 0.000s
I20241112 13:46:47.821381 22287

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  7.523042e+03    0.00e+00    8.34e+04   0.00e+00   0.00e+00  1.00e+04        0    1.86e-03    4.70e-03
   1  6.994160e+03    5.29e+02    3.50e+05   5.05e+01   1.20e-01  6.95e+03        1    3.58e-03    8.29e-03
   2  3.041489e+03    3.95e+03    1.08e+04   2.20e+01   1.00e+00  2.08e+04        1    2.22e-03    1.05e-02
   3  2.940148e+03    1.01e+02    1.55e+04   6.11e+01   9.60e-01  6.25e+04        1    2.35e-03    1.29e-02
   4  2.924055e+03    1.61e+01    5.41e+04   1.04e+02   1.61e-01  4.77e+04        1    2.86e-03    1.58e-02
   5  2.802784e+03    1.21e+02    3.27e+04   5.42e+01   9.20e-01  1.17e+05        1    2.29e-03    1.81e-02
   6  2.902304e+03   -9.95e+01    3.27e+04   7.12e+01  -2.24e+00  5.86e+04        1    9.53e-04    1.90e-02
   7  2.780621e+03    2.22e+01    4.31e+04   4.77e+01   5.77e-01  5.88e+04        1    2.22e-03    2.13e-02
   8  2.759802e+03    2.08e+

I20241112 13:46:52.890115 222879 misc.cc:205] 
Bundle adjustment report
------------------------
I20241112 13:46:52.890148 222879 bundle_adjustment.cc:942] 
    Residuals : 4512
   Parameters : 3393
   Iterations : 101
         Time : 0.25121 [s]
 Initial cost : 1.29126 [px]
   Final cost : 0.77673 [px]
  Termination : No convergence

I20241112 13:46:52.891120 222879 incremental_mapper.cc:160] => Filtered observations: 2
I20241112 13:46:52.891130 222879 incremental_mapper.cc:167] => Filtered images: 0
I20241112 13:46:52.906188 222879 misc.cc:198] 
Registering image #31 (3)
I20241112 13:46:52.906329 222879 incremental_mapper.cc:495] => Image sees 873 / 56886 points
I20241112 13:46:52.951256 222879 misc.cc:205] 
Pose refinement report
----------------------
I20241112 13:46:52.951292 222879 bundle_adjustment.cc:942] 
    Residuals : 1488
   Parameters : 8
   Iterations : 17
         Time : 0.0108321 [s]
 Initial cost : 0.93791 [px]
   Final cost : 0.901184 [px]
  Termination : Convergence

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.456696e+04    0.00e+00    5.09e+04   0.00e+00   0.00e+00  1.00e+04        0    1.05e-02    1.86e-02
   1  1.377394e+04    7.93e+02    7.24e+03   3.58e+01   9.92e-01  3.00e+04        1    1.66e-02    3.53e-02
   2  1.370584e+04    6.81e+01    1.62e+04   5.09e+01   7.84e-01  3.68e+04        1    1.46e-02    4.99e-02
   3  1.367724e+04    2.86e+01    2.40e+04   3.96e+01   5.20e-01  3.68e+04        1    1.49e-02    6.49e-02
   4  1.364191e+04    3.53e+01    2.09e+04   3.10e+01   6.69e-01  3.82e+04        1    1.53e-02    8.02e-02
   5  1.361635e+04    2.56e+01    2.04e+04   2.82e+01   6.20e-01  3.88e+04        1    1.42e-02    9.44e-02
   6  1.359214e+04    2.42e+01    1.91e+04   2.64e+01   6.47e-01  3.98e+04        1    1.41e-02    1.08e-01
   7  1.357055e+04    2.16e+01    1.85e+04   2.56e+01   6.40e-01  4.07e+04        1    1.50e-02    1.24e-01
   8  1.355023e+04    2.03e+

I20241112 13:46:55.080718 222879 misc.cc:205] 
Bundle adjustment report
------------------------
I20241112 13:46:55.080752 222879 bundle_adjustment.cc:942] 
    Residuals : 21292
   Parameters : 15359
   Iterations : 88
         Time : 1.37159 [s]
 Initial cost : 0.827135 [px]
   Final cost : 0.788364 [px]
  Termination : Convergence

I20241112 13:46:55.085572 222879 incremental_mapper.cc:175] => Completed observations: 13
I20241112 13:46:55.089556 222879 incremental_mapper.cc:178] => Merged observations: 0
I20241112 13:46:55.090626 222879 incremental_mapper.cc:160] => Filtered observations: 25
I20241112 13:46:55.090632 222879 incremental_mapper.cc:119] => Changed observations: 0.003569
I20241112 13:46:55.090660 222879 misc.cc:198] 
Global bundle adjustment


...

I20241112 14:26:11.521210 222879 misc.cc:205] 
Bundle adjustment report
------------------------
I20241112 14:26:11.521255 222879 bundle_adjustment.cc:942] 
    Residuals : 5182988
   Parameters : 1260543
   Iterations : 10
         Time : 35.2937 [s]
 Initial cost : 1.06595 [px]
   Final cost : 1.06301 [px]
  Termination : Convergence

I20241112 14:26:13.909215 222879 incremental_mapper.cc:175] => Completed observations: 1889
I20241112 14:26:15.053102 222879 incremental_mapper.cc:178] => Merged observations: 2276
I20241112 14:26:15.593920 222879 incremental_mapper.cc:160] => Filtered observations: 1362
I20241112 14:26:15.594009 222879 incremental_mapper.cc:119] => Changed observations: 0.002133
I20241112 14:26:15.594051 222879 misc.cc:198] 
Global bundle adjustment


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  5.857677e+06    0.00e+00    1.04e+05   0.00e+00   0.00e+00  1.00e+04        0    1.08e+00    1.22e+01
   1  5.853236e+06    4.44e+03    8.74e+02   6.34e+00   1.00e+00  3.00e+04        1    5.85e+00    1.81e+01
   2  5.853171e+06    6.51e+01    7.03e+02   4.05e+00   1.03e+00  9.00e+04        1    2.05e+00    2.01e+01
   3  5.853163e+06    8.42e+00    3.04e+02   3.48e+00   1.02e+00  2.70e+05        1    2.05e+00    2.22e+01
   4  5.853161e+06    1.45e+00    1.77e+02   3.17e+00   1.01e+00  8.10e+05        1    2.10e+00    2.43e+01
   5  5.853161e+06    2.63e-01    6.64e+01   1.82e+00   1.01e+00  2.43e+06        1    2.07e+00    2.63e+01
   6  5.853161e+06    1.27e-02    4.54e+00   4.42e-01   1.02e+00  7.29e+06        1    2.20e+00    2.85e+01
   7  5.853161e+06    1.46e-04    1.70e+00   4.37e-02   1.06e+00  2.19e+07        1    2.28e+00    3.08e+01
   8  5.853161e+06    1.33e-

I20241112 14:26:53.416666 222879 misc.cc:205] 
Bundle adjustment report
------------------------
I20241112 14:26:53.416728 222879 bundle_adjustment.cc:942] 
    Residuals : 5183672
   Parameters : 1259724
   Iterations : 9
         Time : 33.2362 [s]
 Initial cost : 1.06303 [px]
   Final cost : 1.06262 [px]
  Termination : Convergence

I20241112 14:26:55.805832 222879 incremental_mapper.cc:175] => Completed observations: 626
I20241112 14:26:56.964413 222879 incremental_mapper.cc:178] => Merged observations: 842
I20241112 14:26:57.474025 222879 incremental_mapper.cc:160] => Filtered observations: 566
I20241112 14:26:57.474066 222879 incremental_mapper.cc:119] => Changed observations: 0.000785
I20241112 14:26:57.474084 222879 misc.cc:198] 
Global bundle adjustment


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  5.853312e+06    0.00e+00    3.02e+04   0.00e+00   0.00e+00  1.00e+04        0    1.07e+00    1.22e+01
   1  5.851892e+06    1.42e+03    3.30e+02   2.37e+00   1.00e+00  3.00e+04        1    5.86e+00    1.81e+01
   2  5.851889e+06    3.28e+00    1.83e+02   9.41e-01   1.02e+00  9.00e+04        1    2.09e+00    2.02e+01
   3  5.851888e+06    2.28e-01    3.14e+01   5.76e-01   1.03e+00  2.70e+05        1    2.03e+00    2.22e+01
   4  5.851888e+06    3.71e-02    6.79e+00   4.87e-01   1.02e+00  8.10e+05        1    2.15e+00    2.43e+01
   5  5.851888e+06    6.17e-03    2.00e+00   2.78e-01   1.01e+00  2.43e+06        1    2.10e+00    2.64e+01
   6  5.851888e+06    2.97e-04    1.90e+00   6.74e-02   1.03e+00  7.29e+06        1    2.20e+00    2.86e+01
   7  5.851888e+06    3.61e-06    4.27e-01   6.72e-03   1.07e+00  2.19e+07        1    2.23e+00    3.09e+01


I20241112 14:27:33.174319 222879 misc.cc:205] 
Bundle adjustment report
------------------------
I20241112 14:27:33.174360 222879 bundle_adjustment.cc:942] 
    Residuals : 5183406
   Parameters : 1259313
   Iterations : 8
         Time : 31.0861 [s]
 Initial cost : 1.06266 [px]
   Final cost : 1.06253 [px]
  Termination : Convergence

I20241112 14:27:35.593822 222879 incremental_mapper.cc:175] => Completed observations: 234
I20241112 14:27:36.727911 222879 incremental_mapper.cc:178] => Merged observations: 379
I20241112 14:27:37.286350 222879 incremental_mapper.cc:160] => Filtered observations: 221
I20241112 14:27:37.286389 222879 incremental_mapper.cc:119] => Changed observations: 0.000322
I20241112 14:27:37.286417 222879 incremental_mapper.cc:167] => Filtered images: 0
I20241112 14:27:37.588865 222879 timer.cc:91] Elapsed time: 40.830 [minutes]


In [32]:
# dense reconstruction
pycolmap.undistort_images(output_path, output_path, dirname)
pycolmap.patch_match_stereo(output_path)  # requires compilation with CUDA
pycolmap.stereo_fusion(output_path / "dense.ply", output_path)

I20241112 14:27:39.285537 64385 images.h:135]  => Reconstruction with 83 images and 419511 points
I20241112 14:27:39.287904 228750 misc.cc:198] 
Image undistortion
I20241112 14:27:39.291977 228750 undistortion.cc:210] Undistorting image [1/83]
I20241112 14:27:39.777956 228750 undistortion.cc:210] Undistorting image [2/83]
I20241112 14:27:39.778012 228750 undistortion.cc:210] Undistorting image [3/83]
I20241112 14:27:39.778019 228750 undistortion.cc:210] Undistorting image [4/83]
I20241112 14:27:39.778026 228750 undistortion.cc:210] Undistorting image [5/83]
I20241112 14:27:39.778033 228750 undistortion.cc:210] Undistorting image [6/83]
I20241112 14:27:39.778038 228750 undistortion.cc:210] Undistorting image [7/83]
I20241112 14:27:39.778044 228750 undistortion.cc:210] Undistorting image [8/83]
I20241112 14:27:39.778550 228750 undistortion.cc:210] Undistorting image [9/83]
I20241112 14:27:39.778590 228750 undistortion.cc:210] Undistorting image [10/83]
I20241112 14:27:39.778599 228750 un

AttributeError: module 'pycolmap' has no attribute 'patch_match_stereo'

In [ ]:
#model_dir = os.path.abspath('colmap_rec_loftr/0')
#ply_model_path = os.path.abspath('model.ply')
#!colmap model_converter --input_path "{model_dir}" --output_path "{ply_model_path}" --output_type PLY

In [ ]:
#import open3d as o3d

# def visualize_ply(ply_model_path):
#     # Load the .ply file
#     pcd = o3d.io.read_point_cloud(ply_model_path)

#     # Visualize the point cloud
#     o3d.visualization.draw_geometries([pcd])

# visualize_ply('model.ply')

In [ ]:
# !colmap model_merger \
#     --input_path1 ./colmap_rec_loftr/0-1 \
#     --input_path2 ./colmap_rec_loftr/2 \
#     --output_path ./colmap_rec_loftr/total


In [33]:
!colmap gui

qt.qpa.xcb: internal error:  void QXcbWindow::setNetWmStateOnUnmappedWindow() called on mapped window
QOpenGLVertexArrayObject::destroy() failed to restore current context
